In [1]:

%matplotlib widget
#%matplotlib notebook

import src.merger_rates


### ---> plotting
from matplotlib import pyplot as plt

import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap


plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')
plt.rc('font', size=15)
plt.rc('figure', autolayout=True)
plt.rc('axes', titlesize=16, labelsize=17)
plt.rc('lines', linewidth=2, markersize=6)
plt.rc('legend', fontsize=15)
plt.rc('figure')



In [ ]:
M_halo_lst = np.geomspace(1e3, 1e15, 1000)*(0.7)*u.Msun # Halo masses in M_sun/h units

m_1 = 30.*u.Msun
m_2 = 30.*u.Msun
M = m_1 + m_2
eta = m_1*m_2/M**2
binary_mass_params = {"m_1":m_1, "m_2":m_2, "M":M, "eta":eta}

R_ns_over_R_NFW = 0.1

NFW_halo = halo(M_halo_lst, R_ns_over_R_NFW)
merger_rates = rates(NFW_halo, binary_mass_params)




halo_properties = NFW_halo.halo_properties()

print("Halo propoeries: ", halo_properties.keys())

C_lst = halo_properties["Concentration"]
R_NFW_0_lst = halo_properties["R_NFW_0"]
R_vir_lst = halo_properties["R_virial"]
N_ns_lst = halo_properties["N_ns"]
#print(N_ns_lst)
NFW_profiles = halo_properties["NFW_profiles"]
ns_profiles = halo_properties["ns_profiles"]
x_lst = halo_properties["x_lst"]
rho_NFW_0_lst = halo_properties["rho_NFW_0"]
rho_ns_0_lst = halo_properties["rho_ns_0"]
M_star_lst = halo_properties["M_star"]

R_ns_0_lst = R_ns_over_R_NFW*R_NFW_0_lst


cross_section_properties = merger_rates.cross_section()
print("Cross-section properties: ", cross_section_properties.keys())
v_vir_lst = cross_section_properties["v_vir_lst"]
v_dm_lst = cross_section_properties["v_dm_lst"] 
sigma_v_avg = cross_section_properties["sigma_v_avg"]
sigma_reference_200 = cross_section_properties["sigma_reference_200"]


rates_per_halo = merger_rates.rates_per_halo()
print("Per-halo rates: ", rates_per_halo.keys())
PBHPBH_Rate_per_halo = rates_per_halo["PBHPBH_Rate_per_halo"]
PBHPBH_Rate_per_halo_anal = rates_per_halo["PBHPBH_Rate_per_halo_anal"]
PBHNS_Rate_per_halo = rates_per_halo["PBHNS_Rate_per_halo"]


M_c_lst = np.geomspace(1e3, 1e14, 10) 
integarted_rates = merger_rates.integrated_rates(M_c_lst)
print("Integrated rates: ", integarted_rates.keys())
halo_mass_function = integarted_rates["halo_mass_function"]
PBHPBH_rate_lst = integarted_rates["PBHPBH_rate_lst"]
PBHNS_rate_lst = integarted_rates["PBHNS_rate_lst"]






In [ ]:
#C_lst_analytical = 4.67*(M_halo_lst/1e14)**(-0.11)


fig = plt.figure()
ax = fig.gca()

ax.plot(M_halo_lst, C_lst, c = "k")
#ax.plot(M_halo_lst, C_lst_analytical)

ax.set_xscale("log")
ax.set_yscale("log")

ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}]$")
ax.set_ylabel(r"$C[M_\mathrm{h}]$")

ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])


fig = plt.figure()
ax = fig.gca()

ax.plot(M_halo_lst, R_NFW_0_lst, label = r"$R_\mathrm{NFW}^0 [\mathrm{kpc}]$")
ax.plot(M_halo_lst, R_vir_lst, label = r"$R_\mathrm{vir} [\mathrm{kpc}]$")

ax.plot(M_halo_lst, R_ns_0_lst, label = r"$R_\mathrm{ns}^0 [\mathrm{kpc}]$")
ax.plot(M_halo_lst, N_ns_lst, label = r"$N_\mathrm{ns}$")

ax.plot(M_halo_lst, rho_NFW_0_lst, label = r"$\rho^{(0)}_\mathrm{NFW}$")
ax.plot(M_halo_lst, rho_ns_0_lst, label = r"$\rho^{(0)}_\mathrm{ns}$")


ax.set_xscale("log")
ax.set_yscale("log")


ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])
ax.legend()

ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}]$")



fig = plt.figure()
ax = fig.gca()

for indx, x_lst_tmp in enumerate(x_lst):
    if indx%40 == 0:
        ax.plot(x_lst_tmp, rho_NFW_0_lst[indx]*NFW_profiles[indx], c = "k", alpha = 1. - indx/len(x_lst))
        
        ax.plot(x_lst_tmp, rho_ns_0_lst[indx]*ns_profiles[indx], c = "darkred", alpha = 1. - indx/len(x_lst))



ax.set_xscale("log")
ax.set_yscale("log")

ax.set_xlim([np.min(x_lst), np.max(x_lst)])
ax.set_ylim([1e-4, 1e13])


ax.set_xlabel(r"$r/R_\mathrm{NFW}^0$")
ax.set_ylabel(r"$\rho [M_\mathrm{Sun}/\mathrm{kpc}^3]$")

In [ ]:
gamma = 2.
DM_spike = spike(NFW_halo, gamma)

spike_properties = DM_spike.spike_properties()
print("Spike properties: ", spike_properties.keys())
M_SMBH_lst = spike_properties["M_SMBH"].to(u.M_sun)
rho_spike_0_lst = spike_properties["rho_spike_0"].to(u.M_sun/u.kpc**3)
R_spike_0_lst = spike_properties["R_spike"].to(u.pc)


spike_profiles = DM_spike.spike_profiles()
x_lst_spike = spike_profiles["x_list"]
r_lst_spike = spike_profiles["x_list"]*R_spike_0_lst.reshape(len(R_spike_0_lst), 1)
spike_profiles = spike_profiles["spike_profiles"]

fig = plt.figure()
ax = fig.gca()

for indx, r_lst_spike_tmp in enumerate(r_lst_spike):
    if indx%40 == 0:
        ax.plot(r_lst_spike_tmp, rho_spike_0_lst[indx]*spike_profiles[indx], c = "k", alpha = 1. - indx/len(r_lst_spike))


ax.set_xscale("log")
ax.set_yscale("log")

#ax.set_xlim([np.min(x_lst_spike), np.max(x_lst_spike)])
ax.set_ylim([1e10, 1e55])

ax.set_xlabel(r"$r [\mathrm{pc}]$")
ax.set_ylabel(r"$\rho_\mathrm{spike} [M_\mathrm{Sun}/\mathrm{kpc}^3]$")

In [ ]:
M_pbh = 1.*u.M_sun


ref_velo = 200*u.km/u.s
velo_prefac_tmp = (const.G*M_SMBH_lst/R_spike_0_lst)**(1./2.)/ref_velo
velo_prefac = velo_prefac_tmp**(-11./7.)*ref_velo

sigma_v_prefac = 1.4*1e-14*(M_pbh/30./u.M_sun)**2*velo_prefac*u.pc**2


### PBH-PBH
integral = np.trapz(spike_profiles**2*x_lst_spike**2*x_lst_spike**(11./14.), x_lst_spike, axis = 1)



radius_prefac = 4.*np.pi*0.5*R_spike_0_lst**3*rho_spike_0_lst**2/M_pbh**2

integral_prefac = (sigma_v_prefac*radius_prefac).to(1./u.yr)
N_spike_PBH_PBH = integral_prefac*integral

### PBH-NS
ns_profiles = NFW_halo.NS_tilde(x_lst_spike, np.array([R_spike_0_lst, R_ns_0_lst]))
integral = np.trapz(spike_profiles*ns_profiles*x_lst_spike**2*x_lst_spike**(11./14.), x_lst_spike, axis = 1)

radius_prefac = 4.*np.pi*0.5*R_spike_0_lst**3*rho_spike_0_lst*rho_ns_0_lst/M_pbh**2

integral_prefac = (sigma_v_prefac*radius_prefac).to(1./u.yr)
N_spike_PBH_NS = integral_prefac*integral




fig = plt.figure()
ax = fig.gca()

ax.plot(M_SMBH_lst.value, N_spike_PBH_PBH, label = r"PBH-PBH, $f_\mathrm{PBH} = 1$", c = "k", ls = "--")
ax.plot(M_SMBH_lst.value, 1e-6*N_spike_PBH_PBH, label = r"PBH-PBH, $f_\mathrm{PBH} = 10^{-3}$", c = "k")

ax.plot(M_SMBH_lst.value, N_spike_PBH_NS, label = r"PBH-NS, $f_\mathrm{PBH} = 1$", c = "darkred", ls = "--")
ax.plot(M_SMBH_lst.value, 1e-3*N_spike_PBH_NS, label = r"PBH-NS, $f_\mathrm{PBH} = 10^{-3}$", c = "darkred")



ax.set_xscale("log")
ax.set_yscale("log")

ax.legend(ncol = 1, loc = 4)

ax.set_xlim([1e6, 1e8])
#ax.set_ylim([1e-16, 1e-11])

ax.set_xlabel(r"$M_\mathrm{SMBH}[M_\mathrm{Sun}]$")
ax.set_ylabel(r"Merger rate per spike $N_\mathrm{sp}[1/\mathrm{yr}]$")

plt.title(r"$\gamma = 2$")

ax.axhline([1e-2], c = "gray", ls = "--")
ax.axhline([1e-4], c = "gray", ls = "--")



In [ ]:
print(max(R_spike_0_lst/R_NFW_0_lst))
print(max(R_spike_0_lst/R_ns_0_lst))

In [ ]:
fig = plt.figure()
ax = fig.gca()


ax.plot(M_halo_lst, M_star_lst/M_halo_lst, c = "black", ls = "-")

ax.plot(M_halo_lst, 1e-2*M_halo_lst/M_halo_lst, c = "gray", ls = "--")



ax.set_xscale("log")
ax.set_yscale("log")

ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])

ax.set_ylabel(r"$M_\mathrm{star}[M_\mathrm{Sun}/h]/M_\mathrm{h}[M_\mathrm{Sun}/h]$")
ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}/h]$")

In [ ]:
sigma_analytical = 100*(M_halo_lst/1e12)**(1./3.)


print("sigma at v = 200 km/s: ", sigma_reference_200)


fig = plt.figure()
ax = fig.gca()

ax.plot(M_halo_lst, v_vir_lst, label = r"$v_\mathrm{vir}$", c = "k")
ax.plot(M_halo_lst, v_dm_lst, label = r"$v_\mathrm{dm}$", c = "red")
ax.plot(M_halo_lst, sigma_analytical, label = r"$v_\mathrm{dm - anal}$", c = "lightblue")


ax.set_xscale("log")
ax.set_yscale("log")
ax.legend()

ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}/h]$")
ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])


In [ ]:





fig = plt.figure()
ax = fig.gca()

ax.plot(M_halo_lst, PBHPBH_Rate_per_halo, label = r"PBH-PBH, $f_\mathrm{PBH} = 1$", c = "k", alpha = 1)
ax.plot(M_halo_lst, 1e-6*PBHPBH_Rate_per_halo, label = r"PBH-PBH, $f_\mathrm{PBH} = 1e-3$", c = "k", alpha = 0.4)


ax.plot(M_halo_lst, PBHNS_Rate_per_halo, label = r"PBH-NS, $f_\mathrm{PBH} = 1$", c = "red", alpha = 1)
ax.plot(M_halo_lst, 1e-3*PBHNS_Rate_per_halo, label = r"PBH-NS, $f_\mathrm{PBH} = 1e-3$", c = "red", alpha = 0.4)


ax.set_xscale("log")
ax.set_yscale("log")

ax.legend(ncol = 2)

ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])
#ax.set_ylim([1e-16, 1e-11])

ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"Merger rate per halo $\mathcal{R}[1/\mathrm{yr}]$")



fig = plt.figure()
ax = fig.gca()


#ax.plot(M_halo_lst.value*0.7, PBHPBH_Rate_per_halo.value/PBHPBH_Rate_per_halo_anal.value)
ax.plot(M_halo_lst.value, PBHPBH_Rate_per_halo)

ax.plot(M_halo_lst, PBHPBH_Rate_per_halo_anal, ls = "--")

#

ax.set_xscale("log")
ax.set_yscale("log")



ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])
#ax.set_ylim([1e-16, 1e-11])


ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"$\mathcal{R}[1/\mathrm{yr}]$")



In [ ]:
fig = plt.figure()
ax = fig.gca()

ax.scatter(M_c_lst, PBHPBH_rate_lst, c = "k")
ax.scatter(M_c_lst, 1e1*PBHNS_rate_lst, c = "red")

ax.plot(M_c_lst, PBHPBH_rate_lst, label = "PBH-PBH", c = "k")
ax.plot(M_c_lst, 1e1*PBHNS_rate_lst, label = "PBH-NS", c = "red")


ax.set_xscale("log")
ax.set_yscale("log")


ax.legend()


ax.set_xlabel(r"$M_\mathrm{c}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"$\mathrm{Rate}[\mathrm{yr}^{-1}\mathrm{Gpc}^{-3}]$")


ax.axhline([1.], ls = "--", c = "gray")
ax.axhline([.1], ls = "--", c = "gray")
ax.axhline([.01], ls = "--", c = "gray")
ax.axhline([.0001], ls = "--", c = "gray")

ax.axhline([1e-5], ls = "--", c = "gray")

In [ ]:
fig = plt.figure()
ax = fig.gca()

ax.plot(M_halo_lst.value, halo_mass_function, c = "k")



ax.set_xscale("log")
ax.set_yscale("log")



ax.set_xlabel(r"$M_\mathrm{c}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"$\mathrm{d}n/\mathrm{d}ln(M)$")

ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])




In [ ]:
fig = plt.figure()
ax = fig.gca()

ax.plot(M_halo_lst.value, (1./0.7**3)*PBHPBH_Rate_per_halo.value*1e9*halo_mass_function, c = "k")
ax.plot(M_halo_lst.value, (1./0.7**3)*PBHNS_Rate_per_halo.value*1e9*halo_mass_function, c = "red")


#

ax.set_xscale("log")
ax.set_yscale("log")



ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])
#ax.set_ylim([1e-8, 1e0])


ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"$\frac{\mathrm{d}n}{\mathrm{d}\ln M}\mathcal{R}[\mathrm{yr}^{-1}\mathrm{Gpc}^{-3}]$")





In [ ]:
M_halo_lst = np.geomspace(1e3, 1e15, 1000)*(0.7)*u.Msun # Halo masses in M_sun/h units
M_c_lst = np.geomspace(1e3, 1e14, 10) 

m_1 = 30.*u.Msun
m_2 = 30.*u.Msun
M = m_1 + m_2
eta = m_1*m_2/M**2
binary_mass_params = {"m_1":m_1, "m_2":m_2, "M":M, "eta":eta}


############################################################################################
R_ns_over_R_NFW = 0.1
NFW_halo = halo(M_halo_lst, R_ns_over_R_NFW)
merger_rates = rates(NFW_halo, binary_mass_params)


halo_properties = NFW_halo.halo_properties()
rates_per_halo = merger_rates.rates_per_halo()
integarted_rates = merger_rates.integrated_rates(M_c_lst)

PBHPBH_Rate_per_halo_1 = rates_per_halo["PBHPBH_Rate_per_halo"]
PBHNS_Rate_per_halo_1 = rates_per_halo["PBHNS_Rate_per_halo"]
PBHPBH_rate_lst_1 = integarted_rates["PBHPBH_rate_lst"]
PBHNS_rate_lst_1 = integarted_rates["PBHNS_rate_lst"]



############################################################################################
R_ns_over_R_NFW = 0.01
NFW_halo = halo(M_halo_lst, R_ns_over_R_NFW)
merger_rates = rates(NFW_halo, binary_mass_params)


halo_properties = NFW_halo.halo_properties()
rates_per_halo = merger_rates.rates_per_halo()
integarted_rates = merger_rates.integrated_rates(M_c_lst)

PBHPBH_Rate_per_halo_2 = rates_per_halo["PBHPBH_Rate_per_halo"]
PBHNS_Rate_per_halo_2 = rates_per_halo["PBHNS_Rate_per_halo"]
PBHPBH_rate_lst_2 = integarted_rates["PBHPBH_rate_lst"]
PBHNS_rate_lst_2 = integarted_rates["PBHNS_rate_lst"]
############################################################################################








In [ ]:

fig = plt.figure()
ax = fig.gca()



ax.plot(M_halo_lst, PBHPBH_Rate_per_halo_1, label = r"PBH-PBH, $f_\mathrm{PBH} = 1$", c = "red", ls = "-", zorder = 3)
ax.plot(M_halo_lst, 1e-6*PBHPBH_Rate_per_halo_1, label = r"PBH-PBH, $f_\mathrm{PBH} = 10^{-3}$", c = "red", ls = "--", zorder = 3)


ax.plot(M_halo_lst, PBHNS_Rate_per_halo_1, label = r"PBH-NS, $f_\mathrm{PBH} = 1$", c = "k", ls = "-")
ax.plot(M_halo_lst, PBHNS_Rate_per_halo_2, c = "k", ls = "-")
ax.plot(M_halo_lst, 1e-3*PBHNS_Rate_per_halo_1, label = r"PBH-NS, $f_\mathrm{PBH} = 10^{-3}$", c = "k", ls = "--")
ax.plot(M_halo_lst, 1e-3*PBHNS_Rate_per_halo_2, c = "k", ls = "--")

#ax.fill_between(M_halo_lst, PBHNS_Rate_per_halo_1, PBHNS_Rate_per_halo_2, color = "darkred", alpha = 0.3)
#ax.fill_between(M_halo_lst, 1e-3*PBHNS_Rate_per_halo_1, 1e-3*PBHNS_Rate_per_halo_2, color = "darkred", alpha = 0.3)

ratio = (PBHNS_Rate_per_halo_2/PBHNS_Rate_per_halo_1)[0]
gradient_resolution = 100
for indx, scaling in enumerate(np.geomspace(1.1, ratio, gradient_resolution)):
    tranparency = 0.6*(1. - indx/gradient_resolution)

    #print(indx, tranparency)
    ax.plot(M_halo_lst, scaling*PBHNS_Rate_per_halo_1, c = "k", ls = "-", lw = 0.365, alpha = tranparency)
    ax.plot(M_halo_lst, scaling*1e-3*PBHNS_Rate_per_halo_1, c = "k", ls = "-", lw = 0.365, alpha = tranparency)


ax.set_xscale("log")
ax.set_yscale("log")



ax.legend(ncol = 1)

ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])
ax.set_ylim([1e-26, 1e-11])

plt.axvspan(min(M_halo_lst.value), 1e6, color="lightblue", alpha=0.5, zorder = 1)

ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"Merger rate per halo $\mathcal{R}[1/\mathrm{yr}]$")


plt.savefig("Figs/rates_per_halo.pdf")


In [ ]:
fig = plt.figure()
ax = fig.gca()


ax.plot(M_c_lst, PBHNS_rate_lst_1, label = r"PBH-NS, $f_\mathrm{PBH} = 1$", c = "k", ls = "-")
ax.plot(M_c_lst, PBHNS_rate_lst_2, c = "k", ls = "-")
ax.plot(M_c_lst, 1e-3*PBHNS_rate_lst_1, label = r"PBH-NS, $f_\mathrm{PBH} = 10^{-3}$", c = "k", ls = "--")
ax.plot(M_c_lst, 1e-3*PBHNS_rate_lst_2, c = "k", ls = "--")

ax.scatter(M_c_lst, PBHNS_rate_lst_1, color = "k")
ax.scatter(M_c_lst, PBHNS_rate_lst_2, color = "k")
ax.scatter(M_c_lst, 1e-3*PBHNS_rate_lst_1, color = "k")
ax.scatter(M_c_lst, 1e-3*PBHNS_rate_lst_2, color = "k")


ratio = (PBHNS_rate_lst_2/PBHNS_rate_lst_1)[0]
gradient_resolution = 100
for indx, scaling in enumerate(np.geomspace(1.1, ratio, gradient_resolution)):
    tranparency = 0.6*(1. - indx/gradient_resolution)

    ax.plot(M_c_lst, scaling*PBHNS_rate_lst_1, c = "k", ls = "-", lw = 0.365, alpha = tranparency)
    ax.plot(M_c_lst, scaling*1e-3*PBHNS_rate_lst_1, c = "k", ls = "-", lw = 0.365, alpha = tranparency)




ax.plot(M_c_lst, PBHPBH_rate_lst_1, label = r"PBH-PBH, $f_\mathrm{PBH} = 1$", c = "red", ls = "-", zorder = 3)
ax.scatter(M_c_lst, PBHPBH_rate_lst_1, color = "red", zorder = 3)
ax.plot(M_c_lst, 1e-6*PBHPBH_rate_lst_1, label = r"PBH-PBH, $f_\mathrm{PBH} = 10^{-3}$", c = "red", ls = "--", zorder = 3)
ax.scatter(M_c_lst, 1e-6*PBHPBH_rate_lst_1, color = "red", zorder = 3)

ax.set_xscale("log")
ax.set_yscale("log")



#ax.legend(ncol = 1)

ax.set_xlim([min(M_halo_lst.value), 1.2*max(M_c_lst)])
#ax.set_ylim([1e-26, 1e-11])

plt.axvspan(min(M_halo_lst.value), 1e6, color="lightblue", alpha=0.5, zorder = 1)

ax.set_xlabel(r"$M_\mathrm{c}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"$\mathrm{Rate}[\mathrm{yr}^{-1}\mathrm{Gpc}^{-3}]$")

ax.axhline([1.], ls = "--", c = "gray")

plt.savefig("Figs/total_rates.pdf")

In [ ]:
f_PBH = 1e-4

velocity_amplification_factor = (f_PBH**(1./4.))**(-11./7.)
volume_amplification_factor = f_PBH**(-1./2.)



print("Velocity enhancement factor = {:.2e} \n".format(velocity_amplification_factor), 
      "Volume enhancement factor = {:.2e} \n".format(volume_amplification_factor), 
      "Total enhancement factor = {:.2e}".format(velocity_amplification_factor*volume_amplification_factor))


In [ ]:

factor_lst = np.geomspace(1e-4, 1., 10)
#

fig = plt.figure()
ax = fig.gca()


#for indx, val in enumerate(factor_lst):

#    x_lst_test = np.geomspace(1e-5, val*C_lst, 10000).T
    
#    R_C_lst = val*R_NFW_0_lst
#    R_ns_lst_test = R_NFW_0_lst

#    density_integral = np.trapz(x_lst_test**2*NFW_tilde(x_lst_test)*NS_tilde(x_lst_test, np.array([R_C_lst, R_ns_lst_test])), x_lst_test, axis = 1)

#    ax.plot(M_halo_lst.value, density_integral, c = "black")

halo_index = 900
x_lst_test = np.geomspace(1e-10, factor_lst*C_lst[halo_index], 100000).T    
R_C_lst = np.ones(len(factor_lst))*R_NFW_0_lst[halo_index]


R_ns_test = np.ones(len(factor_lst))*R_NFW_0_lst[halo_index]
density_integral = np.trapz(x_lst_test**2*NFW_tilde(x_lst_test)*NS_tilde(x_lst_test, np.array([R_C_lst, R_ns_test])), x_lst_test, axis = 1)

ax.plot(factor_lst, density_integral/density_integral[-1], c = "black")
ax.scatter(factor_lst, density_integral/density_integral[-1], c = "black")


R_ns_test = 1e-1*np.ones(len(factor_lst))*R_NFW_0_lst[halo_index]
density_integral = np.trapz(x_lst_test**2*NFW_tilde(x_lst_test)*NS_tilde(x_lst_test, np.array([R_C_lst, R_ns_test])), x_lst_test, axis = 1)

ax.plot(factor_lst, density_integral/density_integral[-1], c = "red")
ax.scatter(factor_lst, density_integral/density_integral[-1], c = "red")



R_ns_test = 1e-2*np.ones(len(factor_lst))*R_NFW_0_lst[halo_index]
density_integral = np.trapz(x_lst_test**2*NFW_tilde(x_lst_test)*NS_tilde(x_lst_test, np.array([R_C_lst, R_ns_test])), x_lst_test, axis = 1)

ax.plot(factor_lst, density_integral/density_integral[-1], c = "green")
ax.scatter(factor_lst, density_integral/density_integral[-1], c = "green")


ax.plot(factor_lst, 1e2*factor_lst**2, c = "gray")

ax.set_xscale("log")
ax.set_yscale("log")








In [ ]:
H_0 = (cosmo.H0)*u.km/u.Mpc/u.s

R_h_cube_prefac = 2.*const.G*1e8*const.M_sun/H_0**2/0.3/20**3
R_h_prefac = (R_h_cube_prefac.to(u.pc**3))**(1/3)


print(R_h_cube_prefac.to(u.pc**3))
print(R_h_prefac)


velo_sqr_prefac = 2.*const.G*1e8*const.M_sun/const.c**2/R_h_prefac
velo_prefac = np.sqrt(velo_sqr_prefac)

print(velo_sqr_prefac.to(u.m/u.m), velo_prefac.to(u.m/u.m))


timescale_prefac = (H_0/const.c)*(const.c**4/const.G**2)*(1./9.)*(1./1e8/const.M_sun)**2*R_h_prefac**3*velo_prefac**3

print(timescale_prefac.to(u.s/u.s))

In [ ]:
m_PBH = 1.*const.M_sun
m_SMBH = 1e6*const.M_sun

mu = m_PBH*m_SMBH/(m_PBH + m_SMBH)
M = m_PBH + m_SMBH

C = -(96.*2./15.)*(const.G**3/const.c**5)*(1./H_0)*M**2*mu

1./4./C.to(u.pc**4)



In [ ]:
def f_elliptic(ecc):
    return (1. + 73.*ecc**2/24. + 37.*ecc**4/96.)*(1. - ecc**2)**(-7./2.)

one_minus_ecc_lst = np.geomspace(1e-9, 1., 1000)
f_orbit_lst = f_elliptic(1. - one_minus_ecc_lst)

fig = plt.figure()
ax = fig.gca()



ax.plot(one_minus_ecc_lst, f_orbit_lst)

ax.set_yscale("log")
ax.set_xscale("log")

ax.set_xlabel(r"$1 - e$")
ax.set_ylabel(r"$f(e)$")


In [ ]:



class Person:
  def __init__(self, fname, lname):
    self.firstname = fname
    self.lastname = lname

  def printname(self):
    print(self.firstname, self.lastname)
    
class Student(Person):
  pass

#Use the Person class to create an object, and then execute the printname method:

x = Person("John", "Doe")
x.printname()


x = Student("Mike", "Olsen")
x.printname()

x.firstname

In [ ]:


y_prime = np.array([0.142, 0.135, 0.122, 0.103])
x_prime = np.array([0.6, 0.8, 1.0, 1.2])

alpha_gamma_interpolation_function = interpolate.interp1d(x, y)

fig = plt.figure()
ax = fig.gca()

ax.scatter(x, y)
ax.scatter(x_prime, y_prime)

x_interp = np.geomspace(1., 2., 100)
ax.scatter(x_interp, alpha_gamma_interpolation_function(x_interp), c = "gray")